In [1]:
# Imports:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext, DynamicFrame
from awsglue.job import Job
from pyspark.sql.functions import *
from pyspark.sql.types import *

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2,application_1626151071679_0003,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# Context:
# For notebook work, uncomment `context = "notebook"` and comment `context = args["context"]`
context = "notebook"
# context = args["context"]

if context == "notebook":
    tempbucket = "s3://data-quest/temp-directory/"
    Pdatabase = "database-data-quest"
    # For Zeppelin notebook use these:
    glueContext = GlueContext(SparkContext.getOrCreate())
else:
    tempbucket = args["tempbucket"]
    Pdatabase = args["database"]
    # For Glue Job use these:
    sc = SparkContext()
    glueContext = GlueContext(sc)


spark = glueContext.spark_session
job = Job(glueContext)

spark.conf.set("spark.sql.sources.partitionOverwriteMode","dynamic")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# Load Data:
# Source dynamic frame from glue catalog
prDynamicFarme = glueContext.create_dynamic_frame.from_catalog(database=Pdatabase, table_name="csv")
usaDataDynamicFarme = glueContext.create_dynamic_frame.from_catalog(database=Pdatabase, table_name="json")

# Select all data
prDynamicFarme.toDF().registerTempTable("pr")
pr_results = spark.sql("SELECT * FROM pr")

# Select all data
usaDataDynamicFarme.toDF().registerTempTable("usadata")
usadata_results = spark.sql("SELECT * FROM usadata")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
pr_results.show()
usadata_results.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+----+------+-----+--------------+
|        series_id|year|period|value|footnote_codes|
+-----------------+----+------+-----+--------------+
|PRS30006011      |1995|   Q01|  2.6|              |
|PRS30006011      |1995|   Q02|  2.1|              |
|PRS30006011      |1995|   Q03|  0.9|              |
|PRS30006011      |1995|   Q04|  0.1|              |
|PRS30006011      |1995|   Q05|  1.4|              |
|PRS30006011      |1996|   Q01| -0.2|              |
|PRS30006011      |1996|   Q02| -0.3|              |
|PRS30006011      |1996|   Q03| -0.1|              |
|PRS30006011      |1996|   Q04|  0.2|              |
|PRS30006011      |1996|   Q05| -0.1|              |
|PRS30006011      |1997|   Q01|  0.3|              |
|PRS30006011      |1997|   Q02|  0.7|              |
|PRS30006011      |1997|   Q03|  1.0|              |
|PRS30006011      |1997|   Q04|  1.5|              |
|PRS30006011      |1997|   Q05|  0.9|              |
|PRS30006011      |1998|   Q01|  1.6|         

In [5]:
data_struct = ArrayType(StructType([    
        StructField("id", StringType(), True),
        StructField("nation", StringType(), True),
        StructField("year", LongType(), True),
        StructField("year_str", StringType(), True),
        StructField("population", LongType(), True),
        StructField("slug", StringType(), True)
  ]
))

data = usadata_results.withColumn("data_struct", usadata_results.data.cast(data_struct))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
query1 = data.select("data_struct", explode("data_struct.population").alias("population_struct"))
query2 = query1.select("data_struct", "population_struct", explode("data_struct.year").alias("year_struct"))
query2.show(20)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------------+-----------+
|         data_struct|population_struct|year_struct|
+--------------------+-----------------+-----------+
|[[01000US, United...|        327167439|       2018|
|[[01000US, United...|        327167439|       2017|
|[[01000US, United...|        327167439|       2016|
|[[01000US, United...|        327167439|       2015|
|[[01000US, United...|        327167439|       2014|
|[[01000US, United...|        327167439|       2013|
|[[01000US, United...|        325719178|       2018|
|[[01000US, United...|        325719178|       2017|
|[[01000US, United...|        325719178|       2016|
|[[01000US, United...|        325719178|       2015|
|[[01000US, United...|        325719178|       2014|
|[[01000US, United...|        325719178|       2013|
|[[01000US, United...|        323127515|       2018|
|[[01000US, United...|        323127515|       2017|
|[[01000US, United...|        323127515|       2016|
|[[01000US, United...|        323127515|      

In [7]:
from pyspark.sql.functions import mean as _mean, stddev as _stddev, col

query3 = query2.filter(query2["year_struct"].between(2013,2018))
query3.show()

query3 = query3.select(
    _mean(query2["population_struct"].alias('population_mean')),
    _stddev(query2["population_struct"].alias('population_standar_deviation'))
)
query3.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------------+-----------+
|         data_struct|population_struct|year_struct|
+--------------------+-----------------+-----------+
|[[01000US, United...|        327167439|       2018|
|[[01000US, United...|        327167439|       2017|
|[[01000US, United...|        327167439|       2016|
|[[01000US, United...|        327167439|       2015|
|[[01000US, United...|        327167439|       2014|
|[[01000US, United...|        327167439|       2013|
|[[01000US, United...|        325719178|       2018|
|[[01000US, United...|        325719178|       2017|
|[[01000US, United...|        325719178|       2016|
|[[01000US, United...|        325719178|       2015|
|[[01000US, United...|        325719178|       2014|
|[[01000US, United...|        325719178|       2013|
|[[01000US, United...|        323127515|       2018|
|[[01000US, United...|        323127515|       2017|
|[[01000US, United...|        323127515|       2016|
|[[01000US, United...|        323127515|      

In [8]:
querypr1 = pr_results.select("series_id", "year","value").groupBy("series_id", "year").agg(
        sum(pr_results["value"]).alias("Value")
    ).sort("Value", ascending=False)
querypr1.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+----+-----------------+
|        series_id|year|            Value|
+-----------------+----+-----------------+
|PRS32006013      |1995|764.1800000000001|
|PRS32006013      |1996|          751.414|
|PRS32006013      |1997|          746.564|
|PRS32006033      |1995|          745.151|
|PRS32006013      |1998|739.5600000000001|
|PRS32006033      |1997|          735.156|
|PRS30006013      |1998|           733.29|
|PRS32006033      |1996|732.0840000000001|
|PRS32006033      |1998|731.0910000000001|
|PRS30006033      |1998|          730.889|
|PRS31006033      |1998|730.7630000000001|
|PRS31006013      |1998|729.5419999999999|
|PRS30006033      |1997|           728.45|
|PRS30006013      |1997|          727.175|
|PRS31006013      |2000|          726.555|
|PRS31006033      |1997|           724.49|
|PRS31006033      |1999|722.1950000000002|
|PRS31006013      |1999|          722.173|
|PRS30006013      |1999|721.9309999999999|
|PRS31006033      |2000|          721.732|
+----------